In [1]:
# Data visualisation
# By Bibek Gupta

In [2]:
# Load the Pandas libraries with alias 'pd' 
import pandas as pd
import numpy as np
import dask.dataframe as dd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual, IntSlider, SelectMultiple
import warnings
warnings.filterwarnings("ignore",category=UserWarning)
plt.style.use('seaborn-whitegrid')

In [3]:
# Looking for inbuilt styles available
print(plt.style.available)

['Solarize_Light2', '_classic_test_patch', 'bmh', 'classic', 'dark_background', 'fast', 'fivethirtyeight', 'ggplot', 'grayscale', 'seaborn', 'seaborn-bright', 'seaborn-colorblind', 'seaborn-dark', 'seaborn-dark-palette', 'seaborn-darkgrid', 'seaborn-deep', 'seaborn-muted', 'seaborn-notebook', 'seaborn-paper', 'seaborn-pastel', 'seaborn-poster', 'seaborn-talk', 'seaborn-ticks', 'seaborn-white', 'seaborn-whitegrid', 'tableau-colorblind10']


In [4]:
# Constants declaration
max_no_of_companies = 5
default_company = 'Volkswagen'
default_start_year = 2010
max_year = 2021
years_to_consider = [2012, 2013, 2014, 2015, 2016, 2017, 2018]
width_of_bar = 0.75
lables_font_size = 14
xlab = 'No of Volkswagen cars usage'
ylab = 'Year'
title_value = 'Volkswagen cars usage in USA by year'


In [5]:
%%time
# Read the dataset
df = dd.read_csv('used_cars_data.csv', assume_missing=True, low_memory=False, dtype={'bed': 'object',
       'cabin': 'object', 'dealer_zip': 'object'})

Wall time: 216 ms


In [6]:
%%time
# Filter only required columns
df = df[['vin', 'make_name','year']]

Wall time: 2 ms


In [7]:
%%time
# Drop duplicate rows
df.drop_duplicates(subset=['vin'])

Wall time: 5.01 ms


,vin,make_name,year
npartitions=1,,,
,object,object,float64
,...,...,...


In [8]:
# Fill null's with 0 and covert the column to int64
df['year_in_int'] = dd.to_numeric(df['year'], errors='coerce').fillna(0).astype(np.int64)

In [9]:
print (df.columns)

Index(['vin', 'make_name', 'year', 'year_in_int'], dtype='object')


In [10]:
%%time
# Filter just Volkswagen's data
df1 = df[df['make_name'] == default_company]

Wall time: 2.02 ms


In [12]:
# Filter data greater than 2010
df1 = df1[df1['year_in_int'] >= default_start_year]

In [13]:
# Filter comparision data for 2012 to 2018
df_for_compare_volkswagon = df[df['year_in_int'].isin(years_to_consider)]

In [14]:
%%time
# Convert dask dataframe to pandas
pd_df_for_compare_volkswagon = df_for_compare_volkswagon.compute()
pd_df_for_compare_volkswagon.shape

Wall time: 3min 4s


(970221, 4)

In [15]:
# Get top car manufacturers
top_car_makers = pd_df_for_compare_volkswagon['make_name'].value_counts()
# Filter top 4 manufacturers
top_car_makers_other_than_default = top_car_makers.nlargest(max_no_of_companies - 1).index
top_num_array = top_car_makers_other_than_default.to_numpy()
# Adding Volkswagen for comparision
numArray = np.insert(top_num_array, 0, [default_company])
print(numArray)


['Volkswagen' 'Ford' 'Chevrolet' 'Toyota' 'Nissan']


In [16]:
pd_df_for_compare_volkswagon.head()

,vin,make_name,year,year_in_int
2,JF1VA2M67G9829723,Subaru,2016.0,2016
10,ZARBAAC41FM129303,Alfa Romeo,2015.0,2015
12,WBA8B7C53GK368522,BMW,2016.0,2016
36,KMHD84LFXHU271438,Hyundai,2017.0,2017
38,1G1ZD5ST5JF222895,Chevrolet,2018.0,2018


In [17]:
%%time
# Convert dask dataframe to pandas
pdDf = df1.compute()

Wall time: 2min 28s


In [18]:
# Check Shape of data
pdDf.shape

(73894, 4)

In [19]:
pdDf.head(10)

,vin,make_name,year,year_in_int
172,WVGSV7AXXHK031394,Volkswagen,2017.0,2017
263,3VW447AU9JM257055,Volkswagen,2018.0,2018
334,1VWGT7A34HC050446,Volkswagen,2017.0,2017
487,WVWDB7AJ3DW071482,Volkswagen,2013.0,2013
534,3VW4T7AJ6GM344767,Volkswagen,2016.0,2016
535,3VWYT7AU6GM011643,Volkswagen,2016.0,2016
564,3VWD17AJXFM239945,Volkswagen,2015.0,2015
571,WVWSR7AN8KE025364,Volkswagen,2019.0,2019
581,3VW2B7AJ9HM394719,Volkswagen,2017.0,2017
632,3VWD17AJ1EM235068,Volkswagen,2014.0,2014


In [20]:
# Slider Widget
slider = IntSlider(value=default_start_year,
                              min=default_start_year,
                              max=2020,
                              step=1,
                              description='Start Year:',
                              disabled=False,
                              continuous_update=False,
                              orientation='horizontal',
                              readout=True,
                              readout_format='d')
# 1st graph code
def interact_plot_with_start_year(start_year):
    current_visualisation_df = pdDf[pdDf['year_in_int'] >= start_year]
    current_visualisation_df = current_visualisation_df['year_in_int'].value_counts().sort_index()
    plt.rcParams['figure.figsize'] = (15,6)
    barlist = plt.barh(current_visualisation_df.index, current_visualisation_df, width_of_bar)
    # Mark 2015 with different colour
    if start_year <= 2015:
        barlist[2015-start_year].set_color('k')
    ax = plt.gca()
    ymin, ymax = ax.get_ylim()
    custom_ticks = np.linspace(ymin+1, ymax, max_year - start_year + 1, dtype=int)
    ax.set_yticks(custom_ticks)
    ax.set_yticklabels(custom_ticks)
        
    plt.axes().yaxis.grid() # vertical lines
    plt.xlabel(xlab, fontsize=lables_font_size, labelpad=14)
    plt.ylabel(ylab, fontsize=lables_font_size, labelpad=14)
    plt.title(title_value, fontsize=lables_font_size+2, y=1.02)
    plt.savefig('horizontal_bar.png')
interact(interact_plot_with_start_year, start_year = slider)


interactive(children=(IntSlider(value=2010, continuous_update=False, description='Start Year:', max=2020, min=…

<function __main__.interact_plot_with_start_year(start_year)>

In [21]:
# 2nd graph constants
title_value_second = 'Comparison of car usage for the years 2012 to 2018'
y_lab_second = 'No of cars usage'
select_items = sorted(top_num_array.tolist())

# Multi select Widget
multi_select = SelectMultiple(
    options=select_items,
    value=select_items,
    description='Select item:',
    disabled=False
)
# 2nd graph code
def get_selected_items(items):
    list_of_items = list(items)
    list_of_items.insert(0, default_company)
    items = tuple(list_of_items)
    final_sales_count_by_year = []
    plt.rcParams['figure.figsize'] = (15,6)
    for company in items:
        current_df = pd_df_for_compare_volkswagon[pd_df_for_compare_volkswagon['make_name'] == company]
        row_count_by_company = []
        for j in range(len(years_to_consider)):
            temp_df = current_df[current_df['year'] == years_to_consider[j]]
            row, col = temp_df.shape
            row_count_by_company.append(row)
        final_sales_count_by_year.append(row_count_by_company)
    dict_for_bar_chart = {}
    for i in range(len(items)):
        dict_for_bar_chart[items[i]] = final_sales_count_by_year[i]
    compare_df = pd.DataFrame(dict_for_bar_chart, index=years_to_consider)
    my_plot = compare_df.plot.bar(rot=0)
    plt.axes().xaxis.grid()
    plt.xlabel(ylab, fontsize=lables_font_size, labelpad=14)
    plt.ylabel(y_lab_second, fontsize=lables_font_size, labelpad=14)
    plt.title(title_value_second, fontsize=lables_font_size+2, fontweight=1,y=1.02)
    plt.savefig('second_bar.png')
interact(get_selected_items, items = multi_select)


interactive(children=(SelectMultiple(description='Select item:', index=(0, 1, 2, 3), options=('Chevrolet', 'Fo…

<function __main__.get_selected_items(items)>

In [ ]:

# chunk_size = 200000
# batch_no = 1
# for chunk in pd.read_csv("used_cars_data.csv", chunksize=chunk_size):
#     chunk.to_csv('used_cars_data' + str(batch_no) + ".csv", index=False)
#     batch_no += 1
# print("done")

In [ ]:
# %%time
# # print (pd.options.display.max_columns)
# # print (pd.options.display.max_rows)
# df = dd.read_csv('used_cars_data.csv')

In [ ]:


# Read data from file 'filename.csv' 
# (in the same directory that your python process is based)
# Control delimiters, rows, column names with read_csv (see later) 
# %%time
# data = pd.read_csv("used_cars_data1.csv")
# data.head()

In [ ]:
# Preview the first 5 lines of the loaded data 
# data.columns

In [ ]:
# temp = data["make_name"].unique()

In [ ]:
# temp.sort()

In [ ]:
# print(temp)

In [ ]:
# filter = data["make_name"]=="Volkswagen"
# filteredData = data[filter]


In [ ]:
# data["make_name"].value_counts()